In [201]:
# from pyspark.sql import SparkSession
# import sys
# from random import random
# from operator import add

# spark = SparkSession.builder.master("spark://10.1.174.144:7077").appName("PyCount Count").getOrCreate()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
spark = SparkSession \
.builder \
.appName("mongodbtest1") \
.master("spark://10.1.174.144:7077")\
.config("spark.mongodb.read.uri", "mongodb://abhi_1:60000/shardDB.asofashion_2") \
.config("spark.mongodb.write.uri", "mongodb://abhi_1:60000/shardDB.asofashion_clean") \
.config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.13:10.2.2') \
.getOrCreate()

In [2]:
# spark.stop()

In [ ]:
spark

##### Triggers

In [30]:
from pymongo import MongoClient
import pandas as pd

client = MongoClient("149.165.172.75", 60000)
db = client.shardDB

collection = db["asofashion_2"]

try:
    with collection.watch([{"$match": {"operationType": "insert"}}]) as stream:
        for insert_doc in stream:
            lq = LockQueue()
            lq.write_queue([insert_doc])
            
            
except:
    # The ChangeStream encountered an unrecoverable error or the
    # resume attempt failed to recreate the cursor.
    print("Error : occured")

----------------------------------------------------
****
<class 'pymongo.change_stream.CollectionChangeStream'>
Error : occured


In [ ]:
# collection.find_one()

In [42]:
dataFrame = spark.read\
                 .format("mongodb")\
                 .option("connection.uri", "mongodb://abhi_1:60000/")\
                 .option("database", "shardDB")\
                 .option("collection", "asofashion_2")\
                 .load()

In [43]:
dataFrame.count()

140

In [109]:
dataFrame.take(1)

[Row(_id='660f3b689c0dd3afddadd33c', brand_name='Brave Soul', colour=nan, currency='USD', current_price=23.0, gender='male', previous_price=nan, productCode=119545541, productType='Product', product_id=203081399, rrp=42.0, title='Brave Soul cotton check shirt in off white & brown')]

In [101]:
from pyspark.sql import Row

def ser(x):
    x_dict = x.asDict()
    x_dict["title"] = x_dict["title"].lower()+ "_abcd"
    return Row(x_dict)

In [102]:
k = dataFrame.rdd.map(ser)

In [103]:
k.take(1)

[Row(_id='660f3b689c0dd3afddadd33d', brand_name='adidas Originals', colour=nan, currency='USD', current_price=54.72, gender='male', previous_price=99.48, productCode=114983404, productType='Product', product_id=201828775, rrp=nan, title="adidas originals 'sports resort' three stripe wide leg track pants in wonder white_abcd")]

In [110]:
k.count()

140

In [63]:
import pickle

# 0: False:write
# 1: True:read

class LockQueue:

    lock = True

    def get_lock(self):
        return self.lock

    def set_lock(self):
        if(self.lock):
            self.lock = False
            return True
        else:
            return False

    def release_lock(self):
        self.lock=True
        

    def get_read_queue(self):

        if(self.get_lock()):
            with open('queue.pkl', 'rb') as f:
                return pickle.load(f)

    def write_queue(self,list_to_write=[]):

        while True:
            check_lock = self.set_lock()
            if(check_lock):
                break

        with open('queue.pkl', 'wb') as f:
            existing_list = pickle.load(f)
            pickle.dump(existing_list+list_to_write,f)
        
        self.release_lock()
            
        return True

    def flush_queue(self):

        while True:
            check_lock = self.set_lock()
            if(check_lock):
                break

        with open('queue.pkl', 'wb') as f:
            pickle.dump([],f)
        
        self.release_lock()
            
        return True
        
        
        


    
    

In [59]:
f= open('queue.pkl', 'wb')

In [60]:
pickle.dump([],f)

In [61]:
f= open('queue.pkl', 'rb')

In [62]:
import pickle
pickle.load(f)

[]

In [31]:
import queue

In [32]:
queue

<module 'queue' from '/usr/lib/python3.10/queue.py'>